# Analyze results for 3D CGAN
Feb 22, 2021

In [48]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

In [49]:
from ipywidgets import *

In [50]:
%matplotlib widget

In [51]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [52]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/5_3d_cgan/1_main_code/')
import post_analysis_pandas as post


In [53]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


### Read validation data

In [54]:
# bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use

bins=f_transform(bins)   ### scale to (-1,1)
# ### Extract validation data
sigma_lst=[0.5,0.65,0.8,1.1]
labels_lst=range(len(sigma_lst))
bkgnd_dict={}
num_bkgnd=200

for label in labels_lst:
    fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_{0}_train_val.npy'.format(sigma_lst[label])
    print(fname)
    samples=np.load(fname,mmap_mode='r')[-num_bkgnd:][:,0,:,:]
    
    dict_val=post.f_compute_hist_spect(samples,bins)
    bkgnd_dict[str(sigma_lst[label])]=dict_val
    del samples

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.65_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.8_train_val.npy
/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_1.1_train_val.npy


## Read data

In [55]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [56]:
dict1={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [57]:
# parent_dir=u.result
img_size=64
parent_dir=dict1[str(img_size)]
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'202106*')]
n=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(n)

interactive(children=(Dropdown(description='x', options=('20210603_212437_cgan_bs32_one_sigma', '20210615_1546…

In [58]:
result=n.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210615_72613_cgan_bs32_nodes8_lr0.0001


## Plot Losses

In [59]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [60]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
27486,27486.0,46.0,0.194410,0.182606,0.377016,3.580573,14.262688,10.631268,-3.016407,10.893211,-3.120590,NaN,0.050847,2.609994,-3.117037,-3.528220,0.797029,0.0001,0.0001
27487,27487.0,46.0,0.209247,0.284529,0.493776,4.510659,15.370657,10.793213,-2.738630,10.843349,-3.134540,NaN,0.066785,2.526457,-2.094594,-4.486389,0.795881,0.0001,0.0001
27488,27488.0,46.0,0.378111,0.143041,0.521152,2.010068,12.566529,10.543454,-2.858341,10.917457,-2.763858,NaN,0.013007,1.628345,-3.510679,-1.719032,0.806680,0.0001,0.0001
27489,27489.0,46.0,0.091162,0.793402,0.884565,7.973291,18.644222,10.449753,-2.870340,10.899463,-2.962454,NaN,0.221178,3.811656,-0.304857,-7.972461,0.795642,0.0001,0.0001
27490,27490.0,46.0,1.213423,0.124106,1.337530,1.919021,12.980719,10.938882,-2.837388,10.870173,-2.807936,NaN,0.122816,-0.777643,-6.253441,-1.483245,0.796403,0.0001,0.0001
27491,27491.0,46.0,0.186339,0.739184,0.925523,7.614573,18.838276,10.819044,-2.924260,10.828911,-2.883365,NaN,0.404659,5.062234,-0.127313,-7.613245,0.801788,0.0001,0.0001
27492,27492.0,46.0,0.295783,0.186297,0.482080,4.377316,15.403568,10.922866,-2.653668,10.840702,-3.029319,NaN,0.103387,1.803356,-4.786760,-4.344218,0.967158,0.0001,0.0001
27493,27493.0,46.0,0.210103,0.319318,0.529420,2.781960,13.302665,10.503109,-2.990581,10.904605,-2.637160,NaN,0.017596,2.495938,-2.492212,-2.680229,0.796417,0.0001,0.0001
27494,27494.0,46.0,0.136478,0.275233,0.411710,4.291684,15.216784,10.746889,-2.285213,10.763627,-3.059857,NaN,0.178210,2.788497,-1.821008,-4.238590,0.801380,0.0001,0.0001
27495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.735743,-2.345409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [62]:

chi=df_metrics.quantile(q=0.2,axis=0)['hist_chi']
print(chi)
df_metrics[(df_metrics['hist_chi']<=chi)&(df_metrics.epoch>1)].sort_values(by=['hist_chi']).head(10)

-2.8445246696472166


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time,lr_d,lr_g
20101,20101.0,34.0,0.155313,0.232857,0.388170,3.554605,13.940955,10.371447,-2.504447,10.688822,-3.207338,NaN,0.014903,2.912478,-2.054922,-3.470988,0.822406,0.0001,0.0001
17930,17930.0,30.0,0.428393,0.197344,0.625736,1.592858,12.160315,10.521114,-2.758138,10.794972,-3.200840,NaN,0.046342,1.342657,-3.563299,-1.238908,0.800900,0.0001,0.0001
16683,16683.0,28.0,0.798749,0.205819,1.004568,1.828169,12.518588,10.565465,-3.113181,10.805401,-3.200561,NaN,0.124955,0.134789,-5.237815,-1.523185,0.805733,0.0001,0.0001
21882,21882.0,37.0,0.203487,0.207249,0.410736,3.518780,13.798141,10.203991,-2.883409,10.875271,-3.200276,NaN,0.075370,2.152968,-2.586048,-3.431159,0.796641,0.0001,0.0001
19201,19201.0,32.0,0.312851,0.164393,0.477244,3.266400,14.043891,10.733799,-3.163961,10.672949,-3.198747,NaN,0.043693,1.372746,-4.617942,-3.176208,0.802557,0.0001,0.0001
16667,16667.0,28.0,0.174609,0.223220,0.397829,3.688473,14.499527,10.774700,-3.057875,10.610172,-3.197617,NaN,0.036354,2.343224,-2.704945,-3.641310,0.823524,0.0001,0.0001
22947,22947.0,39.0,0.379136,0.128221,0.507357,2.669623,13.388300,10.717468,-2.991698,10.874660,-3.195428,NaN,0.001208,1.020273,-4.660522,-2.440012,0.792551,0.0001,0.0001
19470,19470.0,33.0,0.139573,0.436256,0.575829,5.775671,16.332632,10.434393,-2.534965,10.851583,-3.193240,NaN,0.122568,3.683841,-0.976994,-5.761801,0.812853,0.0001,0.0001
27324,27324.0,46.0,0.197761,0.648955,0.846715,6.665246,17.710907,10.696257,-3.109828,10.750160,-3.190571,NaN,0.349403,5.299510,-0.522144,-6.660387,0.802492,0.0001,0.0001
20099,20099.0,34.0,0.191910,0.463462,0.655373,5.292958,15.969780,10.637912,-2.948456,10.778494,-3.186701,NaN,0.038910,2.993037,-1.350151,-5.284689,0.815620,0.0001,0.0001


In [63]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

<!-- ### Read validation data -->

## Read stored chi-squares for images

In [64]:
## Get sigma list from saved files
flist=glob.glob(result_dir+'/df_processed*')
sigma_lst=[i.split('/')[-1].split('df_processed_')[-1].split('.pkle')[0] for i in flist]
sigma_lst.sort() ### Sorting is important for labels to match !!

labels_lst=np.arange(len(sigma_lst))

In [65]:
sigma_lst,labels_lst

(['0.5', '0.8', '1.1'], array([0, 1, 2]))

In [66]:
### Create a merged dataframe

df_list=[]
for label in labels_lst:
    df=pd.read_pickle(result_dir+'/df_processed_{0}.pkle'.format(str(sigma_lst[label])))
    df[['epoch','step']]=df[['epoch','step']].astype(int)
    df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting
    df_list.append(df)

for i,df in enumerate(df_list):
    df1=df.add_suffix('_'+str(i))
    # renaming the columns to be joined on
    keys=['epoch','step','img_type','label']
    rename_cols_dict={key+'_'+str(i):key for key in keys}
#     print(rename_cols_dict)
    df1.rename(columns=rename_cols_dict,inplace=True) 
    df_list[i]=df1
    
df_merged=reduce(lambda x, y : pd.merge(x, y, on = ['step','epoch','img_type','label']), df_list)

### Get sum of all 4 classes for 3 types of chi-squares
for chi_type in ['chi_1','chi_spec1','chi_1b']:
    keys=[chi_type+'_'+str(label) for label in labels_lst]
#     display(df_merged[keys].sum(axis=1))
    df_merged['sum_'+chi_type]=df_merged[keys].sum(axis=1)
del df_list


### Slice best steps

In [67]:
def f_slice_merged_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',label='all',params_lst=[0,1,2],head=10,epoch_range=[0,None],use_sum=True,display_flag=False):
    ''' View dataframe after slicing
    '''

    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ######### Apply cutoff to keep reasonable chi1 and chispec1
    #### Add chi-square columns to use
    chi_cols=[]
    if use_sum: ## Add sum chi-square columns
        for j in ['chi_1','chi_spec1','chi_1b']: chi_cols.append('sum_'+j)
        
    if label=='all': ### Add chi-squares for all labels
        for j in ['chi_1','chi_spec1','chi_1b']:
            for idx,i in enumerate(params_lst): chi_cols.append(j+'_'+str(idx))
    else: ## Add chi-square for specific label
        assert label in params_lst, "label %s is not in %s"%(label,params_lst)
        label_idx=params_lst.index(label)
        print(label_idx)
        for j in ['chi_1','chi_spec1','chi_1b']: chi_cols.append(j+'_'+str(label_idx))
#     print(chi_cols)
    q_dict=dict(df_merged.quantile(q=cutoff,axis=0)[chi_cols])
    # print(q_dict)
    strg=['%s < %s'%(key,q_dict[key]) for key in chi_cols ]
    query=" & ".join(strg)
    # print(query)
    df=df.query(query)
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
    chis=[i for i in df_merged.columns if 'chi' in i]
    col_list=['label']+chis+['epoch','step']
    if (col_mode=='short'): 
        col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
        col_list=['label']+chi_cols
    df2=df1.head(head)[col_list]
    
    if display_flag: display(df2) # Display df
    
    return df2

# f_slice_merged_df(df_merged,cutoff=0.3,sort_col='sum_chi_1',label=0.65,params_lst=[0.5,0.65,0.8,1.1],use_sum=True,head=2000,display_flag=False,epoch_range=[7,None])

In [68]:
cols_to_sort=np.unique([i for i in df_merged.columns for j in ['chi_1_','chi_spec1_'] if ((i.startswith(j)) or (i.startswith('sum')))])

w=interactive(f_slice_merged_df,df=fixed(df_merged),
cutoff=widgets.FloatSlider(value=0.3, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
use_sum=widgets.Checkbox(value=True),
label=ToggleButtons(options=['all']+sigma_lst), params_lst=fixed(sigma_lst),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=cols_to_sort
)
display(w)

interactive(children=(FloatSlider(value=0.3, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [87]:
df_sliced=w.result
[int(i.split('-')[1]) for i in df_sliced.label.values]

# df_sliced

[]

In [81]:
best_step=[]
df_test=df_merged.copy()
df_test=df_merged[(df_merged.epoch<5000)&(df_merged.epoch>5)]
cut_off=1.0

best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_spec1',label='all',use_sum=True,head=4,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)
best_step.append(f_slice_merged_df(df_test,cutoff=cut_off,sort_col='sum_chi_1b',label='all',use_sum=True,head=2,display_flag=False,epoch_range=[7,None],params_lst=sigma_lst).step.values)

# best_step.append([46669,34281])
best_step=np.unique([i for j in best_step for i in j])
print(best_step)
best_step

[ 8500 10090 12610 12830 14090 14520 25520 25840 26040 26190]


array([ 8500, 10090, 12610, 12830, 14090, 14520, 25520, 25840, 26040,
       26190])

In [83]:
# best_step=[6176]
best_step= [23985,24570,25155,25740,26325,26910,27495]
# best_step=[int(i.split('-')[1]) for i in df_sliced.label.values]
# best_step=np.arange(40130,40135).astype(int)

In [84]:
df_best=df_merged[df_merged.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(3, 61)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [88]:
col_list=['label']+[i for i in df_merged.columns if i.startswith('sum')]
df_best[col_list]

,label,sum_chi_1,sum_chi_spec1,sum_chi_1b
2456,41-24570,0.865809,1.944062,0.757087
2573,43-25740,0.905713,1.761174,0.805013
2690,45-26910,0.817943,1.745933,0.706278


### Interactive plot

In [91]:

def f_plot_hist_spec(df,param_labels,sigma_lst,steps_list,bkg_dict,plot_type,img_size):

    assert plot_type in ['hist','spec','grid','spec_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    for par_label in param_labels:
        df=df[df.step.isin(steps_list)]
#         print(df.shape)
        idx=sigma_lst.index(par_label)
        suffix='_%s'%(idx)
        dict_bkg=bkg_dict[str(par_label)]
        
        for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
            label=row.label+'_'+str(par_label)
            if plot_type=='hist':
                x1=row['hist_bin_centers'+suffix]
                y1=row['hist_val'+suffix]
                yerr1=row['hist_err'+suffix]
                x1=f_invtransform(x1)
                
                plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            if plot_type=='spec':
                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]/np.sqrt(row['num_imgs'+suffix])
                x2=np.arange(len(y2))

                plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
                plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

            if plot_type=='spec_relative':

                y2=row['spec_val'+suffix]
                yerr2=row['spec_sdev'+suffix]
                x2=np.arange(len(y2))

                ### Reference spectrum
                y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
                y=y2/y1
                ## Variance is sum of variance of both variables, since they are uncorrelated

                # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
                yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'+suffix])
                
                plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
                plt.plot(x2, y, marker=marker, linestyle=':',label=label)

            if plot_type=='grid':
                images=np.load(row['fname'+suffix])[:,0,:,:,0]
                print(images.shape)
                f_plot_grid(images[:8],cols=4,fig_size=(8,4))
            
        ### Plot reference data
        if plot_type=='hist':
            x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
            x=f_invtransform(x)
            plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
            plt.title('Pixel Intensity Histogram')
            plt.xscale('symlog',linthreshx=50)
        
        if plot_type=='spec':
            y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
            x=np.arange(len(y))
            plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
            plt.title('Spectrum')
            plt.xlim(0,img_size/2)

        if plot_type=='spec_relative':
            plt.axhline(y=1.0,color='k',linestyle='-.')
            plt.title("Relative spectrum")
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)

    if plot_type in ['hist','spec']:     
        plt.yscale('log')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})


# f_plot_hist_spec(df_merged,[sigma_lst[-1]],sigma_lst,[best_step[0]],bkgnd_dict,'hist')

In [97]:
interact_manual(f_plot_hist_spec,df=fixed(df_best),
                param_labels=SelectMultiple(options=sigma_lst),sigma_lst=fixed(sigma_lst),
                steps_list=SelectMultiple(options=df_best.step.values),
                img_size=fixed(img_size),
                bkg_dict=fixed(bkgnd_dict),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='param_labels', options=('0.5', '0.8', '1.1'), value=()), Sel…

<function __main__.f_plot_hist_spec(df, param_labels, sigma_lst, steps_list, bkg_dict, plot_type, img_size)>

In [95]:
best_step

[23985, 24570, 25155, 25740, 26325, 26910, 27495]

### Delete unwanted stored models
(Since deterministic runs aren't working well )

In [68]:
# # fldr='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210119_134802_cgan_predict_0.65_m2/models'
# fldr=result_dir
# print(fldr)
# flist=glob.glob(fldr+'/models/checkpoint_*.tar')
# len(flist)

In [69]:
# # Delete unwanted stored images
# for i in flist:
#     try:
#         step=int(i.split('/')[-1].split('_')[-1].split('.')[0])
#         if step not in best_step:
# #             print("Found")
# #             print(step)
#             os.remove(i)
#             pass
#         else: 
#             print(step)
# #             print(i)
#     except Exception as e:
# #         print(e)
# #         print(i)
#         pass

In [80]:
best_step

array([2380, 2470, 2480, 2490, 2760, 2870, 2880, 2930, 2950])

In [71]:
# ! du -hs  /global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset2a_3dcgan_4univs_64cube_simple_splicing/norm_1_sig_0.5_train_val.npy